# DIET PROBLEM - PYOMO

*Zuria Bauer Hartwig and David Senén García Hurtado* ( [CAChemE](http://cacheme.org))

Original Problem: [Linear and Integer Programming](https://www.coursera.org/course/linearprogramming) (Coursera Course) - University of Colorado Boulder & University of Colorado System

Based on the Examples from the Optimization Course = [Taller-Optimizacion-Python-Pyomo](https://github.com/CAChemE/Taller-Optimizacion-Python-Pyomo) from [CAChemE.org](http://cacheme.org/optimizacion-programacion-matematica-con-python-pyomo/)

[Neos Guide - The Diet Problem](http://www.neos-guide.org/content/diet-problem)

### SUMMARY

The goal of the diet problem is to select a set of foods we can get at McDonaldsx,
that will satisfy a set of daily nutritional requirement at minimum cost. 
The problem is formulated as a linear program where the objective is to minimize cost 
and the constraints are to satisfy the specified nutritional requirements. 
The diet problem constraints typically regulate the number of calories,
the calories from Fat, Total Fat, Cholesterol, Sodium, 
Carbohydrates, Fiber, Sugar, Proteine, Vitamine A and C,
Calcium and Sodium in the diet. 
While the mathematical formulation is simple, 
the solution may not be palatable! 
The nutritional requirements can be met without regard for taste or variety, 
so consider the output before digging into a meal from an "optimal" menu!


#### 1. Import

In [12]:
# Loading pyomo
from pyomo.environ import *
infinity = float('inf')

#### 2. Data

Let's get the data:

- http://nutrition.mcdonalds.com/getnutrition/nutritionfacts.pdf
- http://www.fastfoodmenuprices.com/mcdonalds-prices/



In [30]:
#Data
!cat data1.dat

#To show the data we can use !cat (or !type for windows) commands

# http://nutrition.mcdonalds.com/getnutrition/nutritionfacts.pdf
# http://www.fastfoodmenuprices.com/mcdonalds-prices/

#Food + Cost

param:  F:                                                 Cal  :=
  "Bacon Clubhouse Burgerx"                                740
  "Bacon Clubhouse Grilled Chicken Sandwichx"              610
  "Bacon Clubhouse Crispy Chicken Sandwichx"               750             
  "McChickenx"                                             370
  "Buffalo Ranch McChickenx"                               370             
  "Bacon Buffalo Ranch McChickenx"                         440             
  "Southern Style Buttermilk Crispy Chicken Sandwichx"     490  
  "Filet-O-Fishx"                                          470             
  "McRibx"                                                 390       
  "Premium McWrap Chicken & Bacon (Crispy)x"               500
  "Premium McWrap Chicken & Bacon (Grilled)x"              640
  "Premium McWrap Chicken & Ranch (Crispy)x" 

### Solving

#### 3. Model

In [31]:
# Creation of a Concrete Model
model = AbstractModel()


#### 4. Sets

In [32]:
#DEFINE SETS

# Products
model.F = Set()
# Nutrients
model.N = Set()

#### 5. Parameters

In [16]:
# DEFINE PARAMETERS

# Calories
model.Cal = Param(model.F, within = PositiveReals)

# Cost
model.a    = Param(model.F, model.N, within = NonNegativeReals)

# Max and Min Cost
model.Cmin = Param(model.N, within = NonNegativeReals, default = 0.0)
model.Cmax = Param(model.N, within = NonNegativeReals, default = infinity)

#### 6. Variables

In [17]:
# Number of servings
model.x = Var(model.F, within = NonNegativeIntegers)


#### 7. Objective

In [18]:
# MAXIMIZE z(calories)
def calories(model):
    return sum(model.Cal[i] * model.x[i] for i in model.F)
model.calories = Objective(rule=calories, sense=maximize)

#### 8. Constrains

In [19]:
#CONSTRAINS

#Max
def cost_max(model, j):
    value = sum(model.a[i,j] * model.x[i] for i in model.F)
    return value <= model.Cmax[j]
model.cost_limit_max = Constraint(model.N, rule=cost_max)

#Min
def cost_min(model, j):
    value = sum(model.a[i,j] * model.x[i] for i in model.F)
    return model.Cmin[j] <= value 
model.cost_limit_min = Constraint(model.N, rule=cost_min)

#### 9. Solution

In [34]:
!cat code1.py

from pyomo.environ import *
infinity = float('inf')

# Creation of a Concrete Model
model = AbstractModel()

#DEFINE SETS
# Products
model.F = Set()
# Nutrients
model.N = Set()

# DEFINE PARAMETERS

# Calories
model.Cal = Param(model.F, within = PositiveReals)

# Cost
model.a    = Param(model.F, model.N, within = NonNegativeReals)

# Max and Min Cost
model.Cmin = Param(model.N, within = NonNegativeReals, default = 0.0)
model.Cmax = Param(model.N, within = NonNegativeReals, default = infinity)

# Number of servings
model.x = Var(model.F, within = NonNegativeIntegers)

# MAXIMIZE z(calories)
def calories(model):
    return sum(model.Cal[i] * model.x[i] for i in model.F)
model.calories = Objective(rule=calories, sense=maximize)


#CONSTRAINS

#Max
def cost_max(model, j):
    value = sum(model.a[i,j] * model.x[i] for i in model.F)
    return value <= model.Cmax[j]
model.cost_limit_max = Constraint(model.N, rule=cost_max)

#Min
def cost_min(model, j):
    value = sum(model.a[i,j] * model.x[

In [35]:
#Get our Solution:

!pyomo solve --solver=glpk code1.py data1.dat

[    0.00] Setting up Pyomo environment
[    0.00] Applying Pyomo preprocessing actions
[    0.00] Creating model
[    0.02] Applying solver
[    0.04] Processing results
    Number of solutions: 1
    Solution Information
      Gap: 0.0
      Status: optimal
      Function Value: 5560.0
    Solver results file: results.yml
[    0.05] Applying Pyomo postprocessing actions
[    0.05] Pyomo Finished


#### 10. Results

In [36]:
#Results
!cat results.yml

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 5560.0
  Upper bound: 5560.0
  Number of objectives: 1
  Number of constraints: 3
  Number of variables: 43
  Number of nonzeros: 85
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 23
      Number of created subproblems: 23
  Error rc: 0
  Time: 0.00930476188659668
# ----------------------------------------------------------
#   Solution Information
# -------------------------------

##### Results

- Bacon Buffalo Ranch McChickenx = 1€
- Hamburguerx = 1.29€